In [ ]:
ROOT_DIR = '/kaggle/input/detectionbnmn-3classes'

In [ ]:
!pip install ultralytics

In [ ]:
!pip install opencv-python

In [ ]:
import os
import numpy as np
import cv2

In [ ]:
import os

from ultralytics import YOLO

mod = "/kaggle/input/yolo11/pytorch/default/1/yolo11l.pt"

#carregar o modelo
models = YOLO(mod) #cria um novo modelo

#Usar o modelo
results = models.train(data=os.path.join(ROOT_DIR, 'teste_file.yaml'), epochs=150, imgsz=1080, batch=8) #treinar o modelo

In [ ]:
model_path = '/kaggle/working/runs/detect/train/weights/best.pt'
model = YOLO(model_path)

metrics = model.val()
metrics.box.map
metrics.box.map50
metrics.box.map75

In [ ]:
from scipy.optimize import linear_sum_assignment

def get_iou(ground_truth, pred):
    # coordinates of the area of intersection.
    ix1 = np.maximum(ground_truth[0], pred[0])
    iy1 = np.maximum(ground_truth[1], pred[1])
    ix2 = np.minimum(ground_truth[2], pred[2])
    iy2 = np.minimum(ground_truth[3], pred[3])
     
    # Intersection height and width.
    i_height = np.maximum(iy2 - iy1 + 1, np.array(0.))
    i_width = np.maximum(ix2 - ix1 + 1, np.array(0.))
     
    area_of_intersection = i_height * i_width
     
    # Ground Truth dimensions.
    gt_height = ground_truth[3] - ground_truth[1] + 1
    gt_width = ground_truth[2] - ground_truth[0] + 1
     
    # Prediction dimensions.
    pd_height = pred[3] - pred[1] + 1
    pd_width = pred[2] - pred[0] + 1
     
    area_of_union = gt_height * gt_width + pd_height * pd_width - area_of_intersection
     
    iou = area_of_intersection / area_of_union
     
    return iou


def draw_bounding_boxes(image, gt_box, pred_box, iou, threshold=0.5, class_correct=True, false_positive=False):
    label = f"IoU: {iou:.2f}"
    
    if false_positive:
        color = (0, 0, 255)
        label = "Detecção incorreta"
        
    elif class_correct and iou >= threshold:
        color = (0, 255, 0)  # Verde
    elif class_correct and iou == 0:
        color = (255, 0, 0)  # Azul
    elif class_correct and iou < threshold:
        color = (0, 255, 255)  # Amarelo
        
    elif class_correct == False:
        color = (0, 0, 255)
        label = "classe incorreta"

    if gt_box is not None:
        cv2.rectangle(image, (int(gt_box[0]), int(gt_box[1])), (int(gt_box[2]), int(gt_box[3])), color, 2)
        cv2.putText(image, label, (int(gt_box[0]), int(gt_box[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    if gt_box is None and pred_box is not None:
    # Desenhar Pred box (Previsão)
        cv2.rectangle(image, (int(pred_box[0]), int(pred_box[1])), (int(pred_box[2]), int(pred_box[3])), color, 2)
        cv2.putText(image, label, (int(pred_box[0]), int(pred_box[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    return image


def load_ground_truth(label_path, image):
    boxes = []
    classes = []
    height, width, _ = image.shape
    with open(label_path, "r") as f:
        for line in f:
            class_id, center_x, center_y, bbox_width, bbox_height = map(float, line.strip().split())
            # Converter para coordenadas absolutas
            x1 = int((center_x - bbox_width / 2) * width)
            y1 = int((center_y - bbox_height / 2) * height)
            x2 = int((center_x + bbox_width / 2) * width)
            y2 = int((center_y + bbox_height / 2) * height)
            boxes.append([x1, y1, x2, y2])
            classes.append(int(class_id))
            
    return boxes, classes

def match_predictions_with_gt(gt_boxes, pred_boxes):
    num_gt = len(gt_boxes)
    num_pred = len(pred_boxes)
    
    # Matriz de IoU (GT x Predições)
    iou_matrix = np.zeros((num_gt, num_pred))

    for i, gt in enumerate(gt_boxes):
        for j, pred in enumerate(pred_boxes):
            iou_matrix[i, j] = get_iou(gt, pred)
    
    # Resolver o problema de associação com Hungarian Algorithm
    gt_indices, pred_indices = linear_sum_assignment(-iou_matrix)  # Negativo porque queremos maximizar IoU
    
    # Identificar correspondências válidas
    matches = []
    unmatched_gt = set(range(num_gt))  # Ground truths sem correspondência
    unmatched_pred = set(range(num_pred))  # Predições sem correspondência

    for gt_idx, pred_idx in zip(gt_indices, pred_indices):
        iou = iou_matrix[gt_idx, pred_idx]
        if iou > 0:  # Apenas associações válidas
            matches.append((gt_idx, pred_idx, iou))
            unmatched_gt.discard(gt_idx)
            unmatched_pred.discard(pred_idx)

    # Adicionar GTs sem correspondência com IoU = 0
    for gt_idx in unmatched_gt:
        matches.append((gt_idx, None, 0.0))

    # Adicionar previsões sem correspondência como falsos positivos
    for pred_idx in unmatched_pred:
        matches.append((None, pred_idx, 0.0))

    return matches

In [ ]:
image_dir = "/kaggle/input/test-dataset/images/test"  # substitua pelo caminho do seu diretório de imagens
label_dir = "/kaggle/input/test-dataset/labels/test"
output_dir = "/kaggle/working/out_720"

image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir)]
os.makedirs(output_dir, exist_ok=True)

for img in image_paths:
   # Carregar imagem
    image = cv2.imread(img)
    image_name = os.path.basename(img)
    
    # Caminho do label correspondente
    label_path = os.path.join(label_dir, os.path.splitext(image_name)[0] + ".txt")
    
    # Carregar caixas Ground Truth
    if not os.path.exists(label_path):
        print(f"Label não encontrado para {image_name}. Pulando...")
        continue
    gt_boxes, gt_classes = load_ground_truth(label_path, image)
    
    # Fazer predições
    results = model.predict(img, save=False, show_conf=False, show_labels=False)
    pred_boxes = results[0].boxes.xyxy.cpu().numpy()  # Predições: [x_min, y_min, x_max, y_max]
    pred_classes = results[0].boxes.cls.cpu().numpy().astype(int)

    matches = match_predictions_with_gt(gt_boxes, pred_boxes)

    for gt_idx, pred_idx, iou in matches:
        if gt_idx is not None:
            gt_box = gt_boxes[gt_idx]
            gt_class = gt_classes[gt_idx]
        else:
            gt_box, gt_class = None, None  # Não foi detectado
    
        if pred_idx is not None:
            pred_box = pred_boxes[pred_idx]
            pred_class = pred_classes[pred_idx]
        else:
            pred_box, pred_class = None, None  # Previsão sem correspondência
    
        # Desenhar caixas na imagem
        if gt_box is not None and pred_box is not None:
            class_correct = gt_class == pred_class
            # Caso normal: GT e predição correspondem
            if class_correct:
                print(f"imagem:{image_name}; iou: {iou}; classe correta")
            else:
                print(f"imagem: {image_name}; IoU: {iou:.2f}; classe incorreta (GT: {gt_class}, Pred: {pred_class}).")
                
            image = draw_bounding_boxes(image, gt_box, pred_box, iou, class_correct=class_correct)
                
        elif gt_box is not None:
            # Caso: GT sem correspondência (IoU = 0)
            print(f"imagem:{image_name}; objeto não detectado. IoU: 0")
            image = draw_bounding_boxes(image, gt_box, [0, 0, 0, 0], 0)  # Caixa fictícia para visualização
        elif pred_box is not None:
            # Caso: Previsão sem correspondência
            print(f"imagem:{image_name}; falso positivo detectado. IoU: 0")
            image = draw_bounding_boxes(image, [0, 0, 0, 0], pred_box, 0, false_positive=True)  # Caixa fictícia para visualização

    save_path = os.path.join(output_dir, image_name)
    cv2.imwrite(save_path, image)
            

In [ ]:
image_dir = "/kaggle/input/test-dataset/images/test"  # substitua pelo caminho do seu diretório de imagens

image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir)]

for img in image_paths:
    model.predict(img, save=True, show_conf=False, show_labels=False)